In [2]:
import numpy as np
import pandas as pd
import gensim
import collections 
import time
import pickle 
from collections import defaultdict
import re 
#from eunjeon import Mecab

In [2]:
#with open("bow_corpus", "rb") as f : 
#    bow_corpus = pickle.load(f)

In [3]:
#print(len(bow_corpus) == 777832)

In [3]:
# 이미 전처리가 된 excel은 pre_processing이 아니라 read_token을 실행하면 된다. 
def read_token(text):
    text = str(text)
    result = text.split()
    result = [i for i in result if type(i) == str]
    
    return result

In [ ]:
flist = ["processed_SNS_1.xlsx","processed_SNS_2.xlsx","processed_SNS_3.xlsx","processed_SNS_4.xlsx","processed_SNS_5.xlsx", "processed_SNS_6.xlsx", "processed_SNS_7.xlsx", "processed_SNS_8.xlsx"]
#collected_df
collected_df = pd.read_excel(flist[0])
#collected_df iteration을 이용하며 총 8개로 분산되어있는 excel파일을 하나의 dataframe으로 가져온다. 
for i in range(1,len(flist)):
    print(collected_df.shape)
    temp_df = pd.read_excel(flist[i])
    collected_df = collected_df.append(temp_df,ignore_index = True)

In [6]:
# read_token하면 자동으로 doc당 형태소 list를 만들 수 있다. 
processed_docs = collected_df["CONTENT"].map(read_token)

In [7]:
def get_tox(processed_docs) :  
    #get index_to_words
    id2word = gensim.corpora.Dictionary(processed_docs)
    #filter_extreme
    id2word.filter_extremes(no_below = 15, no_above = 0.5, keep_n = 100000)
    id2word.compactify()
    
    bow_corpus = [id2word.doc2bow(doc) for doc in processed_docs]
    
    return id2word, bow_corpus

In [8]:
id2word, bow_corpus = get_tox(processed_docs)

In [9]:
with open('bow_corpus.pickle', 'wb') as handle:
    pickle.dump(bow_corpus, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [10]:
def split(a, n):
    k, m = divmod(len(a), n)
    return (a[i * k + min(i, m):(i + 1) * k + min(i + 1, m)] for i in range(n))

In [11]:
with open("idxlist_shuffled", "rb") as f : 
    idx_list = pickle.load(f)

In [12]:
def stream_generator(idx_list, batch_unit = 10):
    print(len(idx_list))
    batch_index = np.array_split(idx_list, batch_unit)
    batch_list = []
    for i in range(len(batch_index)) : 
        temp = [bow_corpus[j] for j in batch_index[i]]
        batch_list.append(temp)
    print(len(batch_list) == batch_unit)
    
    return batch_list

In [13]:
def batch_feed(batch):
    #lda_trained = gensim.models.LdaModel.load("lda_trained.model")
    lda_trained.update(batch)
    lda_trained.save("lda_trained.model")    

In [14]:
batch_list = stream_generator(idx_list)

777832
True


In [15]:
len(batch_list[0])

77784

In [16]:
# Set up log to external log file
import logging
logging.basicConfig(filename='lda_model.log', format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

# Set up log to terminal
#import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

In [17]:
lda_trained =  gensim.models.ldamulticore.LdaMulticore(
                       corpus = batch_list[0],
                       num_topics=15,
                       id2word=id2word,
                       workers=3, # Num. Processing Cores - 1
                       passes=5,
                       chunksize = 100,
                       eval_every = 1,
                       per_word_topics=True)
lda_trained.save('lda_trained.model')

In [18]:
lda_trained.update(batch_list[1])

In [19]:
for i in range(2,len(batch_list)) :
    print("{0}th batch is now updating...".format(i+1))
    lda_trained.update(batch_list[i])
    print("Done!")
    

3th batch is now updating...
Done!
4th batch is now updating...
Done!
5th batch is now updating...
Done!
6th batch is now updating...
Done!
7th batch is now updating...
Done!
8th batch is now updating...
Done!
9th batch is now updating...
Done!
10th batch is now updating...
Done!


In [41]:
for idx, topic in lda_trained.print_topics(16, num_words = 17):
    print('Topic: {} Word: {}'.format(idx, topic))

Topic: 0 Word: 0.055*"제품" + 0.044*"새" + 0.042*"남" + 0.037*"판매" + 0.033*"▶" + 0.032*"원" + 0.031*"개" + 0.029*"가능" + 0.024*"비" + 0.024*"천" + 0.022*"택배" + 0.021*"수량" + 0.021*"별도" + 0.020*"직거래" + 0.012*"완료" + 0.012*"세트"
Topic: 1 Word: 0.015*"적" + 0.011*"건강" + 0.009*"합니다" + 0.008*"때문" + 0.008*"인" + 0.008*"먼지" + 0.007*"주" + 0.007*"경우" + 0.006*"미세" + 0.006*"많" + 0.006*"치료" + 0.006*"발생" + 0.005*"성" + 0.005*"질환" + 0.005*"시" + 0.005*"물질"
Topic: 2 Word: 0.013*"대" + 0.009*"성" + 0.008*"곳" + 0.008*"역" + 0.007*"눈" + 0.006*"인" + 0.006*"피부" + 0.005*"없애" + 0.005*"코" + 0.005*"잘" + 0.005*"볼" + 0.004*"턱" + 0.004*"라인" + 0.004*"목" + 0.004*"관리" + 0.004*"시술"
Topic: 3 Word: 0.038*"먹" + 0.015*"맛있" + 0.015*"맛" + 0.010*"원" + 0.009*"시" + 0.008*"카페" + 0.007*"맛집" + 0.007*"주문" + 0.006*"배" + 0.006*"주" + 0.006*"고기" + 0.005*"식물" + 0.005*"메뉴" + 0.005*"선물" + 0.005*"음식" + 0.005*"넣"
Topic: 4 Word: 0.011*"·" + 0.010*"환경" + 0.009*"적" + 0.009*"사업" + 0.008*"’" + 0.007*"지원" + 0.007*"인" + 0.006*"원" + 0.006*"지역" + 0.006*"도시" + 0.006